In [ ]:
import os 
import numpy as np
import cv2
from glob import glob
from PIL import Image
import imutils
import gc
import tensorflow as tf

In [ ]:
image_overlap_train = np.load('./data_split/overlap/image_overlap_train.npy')
image_overlap_val = np.load('./data_split/overlap/image_overlap_val.npy')

f_val = open('./data_split/overlap/txt_overlap_val.txt')
txt_overlap_val = []
for line in f_val:
    line = line.strip('\n')
    txt_overlap_val.append(line)
    
f_train = open('./data_split/overlap/txt_overlap_train.txt')
txt_overlap_train = []
for line in f_train:
    line = line.strip('\n')
    txt_overlap_train.append(line)

txt_overlap_train = np.array(txt_overlap_train)
txt_overlap_val = np.array(txt_overlap_val)
# image_overlap_train = np.array(image_overlap_train)
# image_overlap_val = np.array(image_overlap_val)

print(image_overlap_train.shape)
print(image_overlap_val.shape)
print(txt_overlap_train.shape)
print(txt_overlap_val.shape)

# save train tfrecords

In [ ]:
# TFRecords 檔案名稱
tfrecords_filename = './data_split/overlap/image_overlap_train.tfrecords'

# 建立 TFRecordWriter
writer = tf.python_io.TFRecordWriter(tfrecords_filename)    

In [ ]:
for i in range(len(image_overlap_train)):
    example = tf.train.Example(features=tf.train.Features(feature={
        'image_train':tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_overlap_train[i].astype(np.float32).tostring()])),
        'txt_train':tf.train.Feature(bytes_list=tf.train.BytesList(value=[bytes(txt_overlap_train[i], encoding='utf8')]))
    }))
    
    writer.write(example.SerializeToString())

# 關閉 TFRecordWriter
writer.close()

# open train tfrecords

In [ ]:
tfrecords_filename = './data_split/overlap/image_overlap_train.tfrecords'
filename_queue = tf.train.string_input_producer([tfrecords_filename], num_epochs=50)

# 建立 TFRecordReader
reader = tf.TFRecordReader()

# 讀取 TFRecords 的資料
_, serialized_example = reader.read(filename_queue)

features = tf.parse_single_example(
    serialized_example,
    features={
      'image_train': tf.FixedLenFeature([], tf.string),
      'txt_train': tf.FixedLenFeature([], tf.string)
      })

In [ ]:
image_train = tf.decode_raw(features['image_train'], tf.float32)

In [ ]:
image_trainimage_train